In [1]:
import cv2
import mediapipe

## Hand Landmarks Mediapipe
**Hand has 21 Point landmarks**
<img src="Images/hand_pose.png">


### Detecting Hands and Drawing Connections

In [2]:
cap = cv2.VideoCapture(0)

# Calling mediapipe for hands
mphands = mediapipe.solutions.hands

# it takes the image flipped (as if it's selfie or front camera) --> we use cv2.flip(img,1)
hands = mphands.Hands(static_image_mode = False, max_num_hands=2)

# To draw connections between points
mpDraw = mediapipe.solutions.drawing_utils

while True:
    ret, frame = cap.read()
    if ret == True:
        
        # Mediapipe only works on RGB
        frame = cv2.flip(frame, 1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Applying Mediapipe to the images
        results = hands.process(frame_rgb)

        # Drawing Circles and Connections when find a hand
        if results.multi_hand_landmarks:
            for hand in results.multi_hand_landmarks:
                mpDraw.draw_landmarks(frame, hand, mphands.HAND_CONNECTIONS)
        
        
                ## Printing Landmarks Position (for further Programming)
                # for (idd, lm) in enumerate(hand.landmark):
                    # print(idd, lm)
                
        # Showing the image
        frame = cv2.flip(frame, 1)
        cv2.imshow('frame', frame)
        
        
        key = cv2.waitKey(1)
        if key == 27:  # ESC
            break
    else: break
    
cv2.destroyAllWindows()
cap.release()

### Detecting Type of hand (left/right)

In [41]:
cap = cv2.VideoCapture(0)

# Calling mediapipe for hands
mphands = mediapipe.solutions.hands

# it takes the image flipped (as if it's selfie or front camera)
# if it's not the case use cv2.flip(img, 1)
hands = mphands.Hands(static_image_mode = False, max_num_hands=2)

# To draw connections between points
mpDraw = mediapipe.solutions.drawing_utils

while True:
    ret, frame = cap.read()
    if ret == True:
        
        handType = ""
        # Mediapipe only works on RGB
        frame = cv2.flip(frame, 1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        
        # Applying Mediapipe to the images
        results = hands.process(frame_rgb)

        # Drawing Circles and Connections when find a hand
        if results.multi_hand_landmarks:
            for hand in results.multi_hand_landmarks:
                mpDraw.draw_landmarks(frame, hand, mphands.HAND_CONNECTIONS)
                
            # HAND TYPE DETECTOR
            for hand in results.multi_handedness:
                handType=hand.classification[0].label
        
        
        # Showing the image
        frame = cv2.flip(frame,1)
        cv2.putText(frame, handType, (10,20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 1)
        
        cv2.imshow('frame', frame)
        
        
        key = cv2.waitKey(1)
        if key == 27:  # ESC
            break
    else: break
    
cv2.destroyAllWindows()
cap.release()

### Moving Mouse using Hand
<img src="Images/hand_pose.png">


#### Function to check which fingers are up

In [50]:
# We will use TIPs of Fingers --> Landmarks [4,8,12,16,20]
# if all fingers are closed except (2nd Finger --> TIP[8]) then it moves the mouse
# if all fingers are closed except (thumb finger --> TIP[4]) then it clicks!

#### NOTE:
# 1- To check if finger is up we compare using Y-axis
# 2- To check if Thumbs is up we compare using X-axis

def fingers(landmarks, handtype):
    """
    landmarks: list of (id,position_x,position_y) for all landmarks
    """
    fingerTips = []   # To store 5 sets of 1s or 0s
    tipIds = [4,8,12,16,20]  # indexes for tips of each finger
    
    # Checking Thumbs UP (x-axis)
    if handtype == 'Left':
        if landmarks[tipIds[0]].x > landmarks[tipIds[0]-1].x:  # X landmark[4] > X landmark[3]  --> THUMBS
            fingerTips.append(1)
        else:
            fingerTips.append(0)
            
    elif handtype == 'Right':
        if landmarks[tipIds[0]].x < landmarks[tipIds[0]-1].x:  # X landmark[4] < X landmark[3]  --> THUMBS
            fingerTips.append(1)
        else:
            fingerTips.append(0)
        
        
    # check if all fingers are up --> [2] means y axis (in all other fingers we check y axis)
    for id in range(1,5):
        if landmarks[tipIds[id]].y < landmarks[tipIds[id] - 3].y:  # less than
            fingerTips.append(1)
        else:
            fingerTips.append(0)
            
    return fingerTips

#### Function to get position of finger on the screen

In [69]:
import numpy as np

def get_screen_position(finger, frame_h, frame_w):
    
    wScr, hScr = autopy.screen.size() # Width, Height of the screen

    x1 = landmarks[4].x * frame_w
    y1 = landmarks[4].y * frame_h
        
    x = np.interp(x1, (0,640), (0,wScr))  # converts width of the frame into screen width
    y = np.interp(y1, (0,480), (0,hScr))  # converts height of the frame into screen height
    
    return x,y

#### Collecting them, and show results

In [74]:

# To solve the problem of mouse vibration
pX, pY = 0, 0   # prvious x and y locations  
cX, cY = 0, 0   # current x and y locations

cap = cv2.VideoCapture(0)

# Calling mediapipe for hands
mphands = mediapipe.solutions.hands

# if it's not the case use cv2.flip(img, 1)
hands = mphands.Hands(static_image_mode = False, max_num_hands=1)

# To draw connections between points
mpDraw = mediapipe.solutions.drawing_utils


while True:
    ret, frame = cap.read()
    h, w, c = frame.shape
    if ret == True:
        
        # Mediapipe only works on RGB
        frame = cv2.flip(frame, 1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Applying Mediapipe to the images
        results = hands.process(frame_rgb)

        # Drawing Circles and Connections when find a hand
        if results.multi_hand_landmarks:
            # HAND TYPE DETECTOR
            for hand in results.multi_handedness:
                handType=hand.classification[0].label
                
            for hand in results.multi_hand_landmarks:
                mpDraw.draw_landmarks(frame, hand, mphands.HAND_CONNECTIONS)
            
                # Getting Locations for each landmark
                landmarks = results.multi_hand_landmarks[0].landmark
                fingers_up = fingers(landmarks, handType)
                
                # CHECKING IF POINTER IS UP (MOVING THE MOUSE)
                if fingers_up[1] == 1 and fingers_up[2] == 0 and fingers_up[3] == 0 and fingers_up[4] == 0:
                    x, y = get_screen_position(landmarks[4], h, w)
                    
                    # to solve vibration of mouse
                    cX = pX + (x - pX) / 7
                    cY = pY + (y - pY) / 7
                    pX, pY = cX, cY
                    
                    # moving mouse
                    autopy.mouse.move(cX, cY)
                
                # CHECKING IF THUMBS IS ONLY UP (CLICK!)
                elif fingers_up[0] == 1 and sum(fingers_up) == 1:
                    autopy.mouse.click()  # left click
                                     
        
        # Showing the image
        frame = cv2.flip(frame,1)        
        cv2.imshow('frame', frame)
        
        
        key = cv2.waitKey(1)
        if key == 27:  # ESC
            break
    else: break
    
cv2.destroyAllWindows()
cap.release()

## Pose Mediapipe Landmarks
<img src="Images/pose_landmarks.png">

In [76]:
import cv2
import mediapipe
import numpy as np

# Calling mediapipe for pose
mp_drawing = mediapipe.solutions.drawing_utils
mp_pose = mediapipe.solutions.pose

pose = mp_pose.Pose()


cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret,frame = cap.read()
    
    if ret == True:
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        results = pose.process(image)
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Render Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(255,0,0), thickness = 1))
        
        cv2.imshow('Mediapipe', image)
        
        key = cv2.waitKey(1)
        if key == 27:  # ESC
            break
    else: break
    
cv2.destroyAllWindows()
cap.release()
        
